### Sequential Chain

##### Boilerplate code

In [ ]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


In [20]:
# First chain
first_prompt = ChatPromptTemplate.from_template(
    "How much is {a} + {b}. Answer only with the result\nResult: "
)
first_chain = first_prompt | google_llm | StrOutputParser()

# Second chain
second_prompt = ChatPromptTemplate.from_template(
    "How much is {a} times {c}"
)
second_chain = second_prompt | google_llm | StrOutputParser()


##### Option 1 - Recommended - you get all intermediate outputs

In [ ]:
from langchain.schema.runnable import RunnablePassthrough


complete_chain = (
    RunnablePassthrough.assign(c=first_chain)
    |
    RunnablePassthrough.assign(d=second_chain)
)

result = complete_chain.invoke({'a': 2, 'b': 3})
print(result)



{'a': 2, 'b': 3, 'c': '5', 'd': '2 times 5 is 10.'}


##### Option 2 - Use only if you want the final output

In [ ]:
from operator import itemgetter

# Complete sequential chain using LCEL
complete_chain = (
    {
        "a": itemgetter("a"),
        "b": itemgetter("b"),
        "c": first_chain
    }
    | second_chain | PromptTemplate.from_template("Just print the result of this calculation and nothing else: {abc}") | google_llm | StrOutputParser()
)

result = complete_chain.invoke({'a': 2, 'b': 3})
print(result)

10


In [34]:
company_name_prompt = ChatPromptTemplate.from_template(
    "You are a company name generator from the given product idea. Answer with max two words as output. Here is the product idea: {product_idea}"
)

company_name_chain = company_name_prompt | google_llm | StrOutputParser()

description_prompt = ChatPromptTemplate.from_template(
    "You are a company name description generator from the give company name and idea. Describe the company in two lines. Here is the product idea: {product_idea} and company name: {company_name}"
)

description_chain = description_prompt | google_llm | StrOutputParser()

motto_prompt = ChatPromptTemplate.from_template(
    "You are a company motto generator from the give company name, idea and description. Motto should be catchy and short. Just give me one. Here is the product idea: {product_idea} and company name: {company_name} and company description {company_description}"
)

motto_chain = motto_prompt | google_llm | StrOutputParser()


In [64]:
from pprint import pprint

complete_chain = (
    RunnablePassthrough.assign(company_name=company_name_chain) 
    |
    RunnablePassthrough.assign(company_description=description_chain)
    |
    RunnablePassthrough.assign(motto=motto_chain)
)

res = complete_chain.invoke({"product_idea": "An online platform where people can upload and share photos"})
print(res)

{'product_idea': 'An online platform where people can upload and share photos', 'company_name': 'Pixel Share', 'company_description': 'Pixel Share is your vibrant online community for visual storytelling.\nUpload, share, and discover captivating photos from around the world, connecting with fellow photography enthusiasts.', 'motto': 'Pixel Share: Share the View.'}


##### Simple Sequential chain

In [47]:
# **All chains use same variable name "text"**
chain1 = PromptTemplate.from_template("Just answer 'One' for this: {text}") | google_llm | StrOutputParser()

chain2 = PromptTemplate.from_template("Just answer 'Two' for this: {text}") | google_llm | StrOutputParser()

chain3 = PromptTemplate.from_template("Just answer 'Three' for this: {text}") | google_llm | StrOutputParser()

chain4 = PromptTemplate.from_template("Just print what you see: {text}") | google_llm | StrOutputParser()

# **Simple sequential pipe**
chain = chain1 | chain2 | chain3 | chain4

# **Execute**
result = chain.invoke({"text": "AI is powerful technology"})
print(result)

Three


##### RunnableSequence

In [ ]:
from langchain_core.runnables import RunnableSequence

prompt_template = ChatPromptTemplate.from_template("Answer all my questions briefly. Here is my {question}")

chain = prompt_template | google_llm | StrOutputParser()

chain = RunnableSequence(first=prompt_template, middle=[google_llm], last=StrOutputParser())

chain = prompt_template.pipe(google_llm).pipe(StrOutputParser())

chain.invoke("Who is Ana de Armas?")

'Ana de Armas is a Cuban-Spanish actress.'

In [ ]:
# if any(for):
#     print("Present")
# else:
#     print("Not present")

TypeError: 'bool' object is not iterable

In [75]:
any(x == 2 for x in [1,2,3])

True